<a href="https://colab.research.google.com/github/fragenabhishek/ML/blob/main/neural_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import pandas as pd
import numpy as np
import sklearn


switch from tensor flow to theano

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "theano"
import keras.backend
keras.backend.set_image_dim_ordering('th')

In [ ]:
from google.colab import files
uploaded =files.upload()

Saving pimaindiansdiabetescsv.zip to pimaindiansdiabetescsv.zip


In [ ]:

names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
df=pd.read_csv("pimaindiansdiabetescsv.zip",names=names)

In [ ]:
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
df[df['glucose_concentration'] == 0]

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [ ]:
# Preprocess the data, mark zero values as NaN and drop
columns = ['glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']

for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
    
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
# Drop rows with missing values
df.dropna(inplace=True)

# summarize the number of rows and columns in df
df.describe()

,n_pregnant,glucose_concentration,blood_pressuer (mm Hg),skin_thickness (mm),serum_insulin (mu U/ml),BMI,pedigree_function,age,class
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
# Convert dataframe to numpy array
dataset = df.values
print(dataset.shape)

(392, 9)


In [ ]:
# split into input (X) and an output (Y)
X = dataset[:,0:8]
Y = dataset[:, 8].astype(int)

In [ ]:
print(X.shape)
print(Y.shape)
print(Y[:5])

(392, 8)
(392,)
[0 1 1 1 1]


In [ ]:
# Normalize the data using sklearn StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)

In [ ]:
print(scaler)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [ ]:
# Transform and display the training data
X_standardized = scaler.transform(X)

data = pd.DataFrame(X_standardized)
data.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [ ]:
# import necessary sklearn and keras packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [ ]:
# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

model = create_model()
print(model.summary())

W0723 14:47:08.158198 139647676114816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0723 14:47:08.217692 139647676114816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0723 14:47:08.227235 139647676114816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0723 14:47:08.271329 139647676114816 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0723 14:47:08.305397 139647676114816 deprecation_wra

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# Do a grid search for the optimal batch size and number of epochs
# import necessary packages
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, verbose = 1)

# define the grid search parameters
batch_size = [10, 20, 40]
epochs = [10, 50, 100]

# make a dictionary of the grid search parameters
param_grid = dict(batch_size=batch_size, epochs=epochs)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=10, epochs=10 ........................................
Epoch 1/10
261/261 [==============================] - 3s 10ms/step - loss: 0.6292 - acc: 0.6897
Epoch 2/10
261/261 [==============================] - 0s 178us/step - loss: 0.4808 - acc: 0.6973
Epoch 3/10
261/261 [==============================] - 0s 163us/step - loss: 0.4402 - acc: 0.6973
Epoch 4/10
261/261 [==============================] - 0s 175us/step - loss: 0.4254 - acc: 0.6973
Epoch 5/10
261/261 [==============================] - 0s 179us/step - loss: 0.4141 - acc: 0.8084
Epoch 6/10
261/261 [==============================] - 0s 174us/step - loss: 0.4059 - acc: 0.8123
Epoch 7/10
261/261 [==============================] - 0s 168us/step - loss: 0.3998 - acc: 0.8199
Epoch 8/10
261/261 [==============================] - 0s 179us/step - loss: 0.3969 - acc: 0.8161
Epoch 9/10
261/261 [==============================] - 0s 182us/step - loss: 0.3965 - acc: 0.81

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 2s 8ms/step - loss: 0.6398 - acc: 0.6475
Epoch 2/10
261/261 [==============================] - 0s 180us/step - loss: 0.5226 - acc: 0.6552
Epoch 3/10
261/261 [==============================] - 0s 185us/step - loss: 0.4920 - acc: 0.6552
Epoch 4/10
261/261 [==============================] - 0s 173us/step - loss: 0.4815 - acc: 0.7011
Epoch 5/10
261/261 [==============================] - 0s 175us/step - loss: 0.4743 - acc: 0.7893
Epoch 6/10
261/261 [==============================] - 0s 179us/step - loss: 0.4617 - acc: 0.8123
Epoch 7/10
261/261 [==============================] - 0s 180us/step - loss: 0.4555 - acc: 0.8123
Epoch 8/10
261/261 [==============================] - 0s 175us/step - loss: 0.4482 - acc: 0.8008
Epoch 9/10
261/261 [==============================] - 0s 176us/step - loss: 0.4352 - acc: 0.8123
Epoch 10/10
131/131 [==============================] - 1s 7ms/step
[CV] ............ batch_size=10, epochs=10, score=0.771, total

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    8.2s remaining:    0.0s


...

Epoch 1/100
262/262 [==============================] - 3s 10ms/step - loss: 0.6448 - acc: 0.6336
Epoch 2/100
262/262 [==============================] - 0s 180us/step - loss: 0.5359 - acc: 0.6527
Epoch 3/100
262/262 [==============================] - 0s 183us/step - loss: 0.5114 - acc: 0.6527
Epoch 4/100
262/262 [==============================] - 0s 185us/step - loss: 0.4982 - acc: 0.7252
Epoch 5/100
262/262 [==============================] - 0s 206us/step - loss: 0.4920 - acc: 0.7786
Epoch 6/100
262/262 [==============================] - 0s 188us/step - loss: 0.4869 - acc: 0.7634
Epoch 7/100
262/262 [==============================] - 0s 187us/step - loss: 0.4807 - acc: 0.7595
Epoch 8/100
262/262 [==============================] - 0s 190us/step - loss: 0.4775 - acc: 0.7595
Epoch 9/100
262/262 [==============================] - 0s 201us/step - loss: 0.4699 - acc: 0.7863
Epoch 10/100
262/262 [==============================] - 0s 195us/step - loss: 0.4635 - acc: 0.7786
Epoch 11/100
262/262

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   59.0s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 3s 10ms/step - loss: 0.6722 - acc: 0.6705
Epoch 2/10
261/261 [==============================] - 0s 115us/step - loss: 0.5611 - acc: 0.7318
Epoch 3/10
261/261 [==============================] - 0s 106us/step - loss: 0.4398 - acc: 0.7931
Epoch 4/10
261/261 [==============================] - 0s 106us/step - loss: 0.4075 - acc: 0.8046
Epoch 5/10
261/261 [==============================] - 0s 113us/step - loss: 0.3932 - acc: 0.8084
Epoch 6/10
261/261 [==============================] - 0s 105us/step - loss: 0.3800 - acc: 0.8199
Epoch 7/10
261/261 [==============================] - 0s 118us/step - loss: 0.3745 - acc: 0.8276
Epoch 8/10
261/261 [==============================] - 0s 111us/step - loss: 0.3798 - acc: 0.8084
Epoch 9/10
261/261 [==============================] - 0s 123us/step - loss: 0.3798 - acc: 0.8161
Epoch 10/10
131/131 [==============================] - 1s 8ms/step
[CV] ............ batch_size=20, epochs=10, score=0.710, tota

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  3.0min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/100
392/392 [==============================] - 4s 10ms/step - loss: 0.6057 - acc: 0.6633
Epoch 2/100
392/392 [==============================] - 0s 227us/step - loss: 0.4934 - acc: 0.6684
Epoch 3/100
392/392 [==============================] - 0s 229us/step - loss: 0.4784 - acc: 0.7245
Epoch 4/100
392/392 [==============================] - 0s 229us/step - loss: 0.4652 - acc: 0.7934
Epoch 5/100
392/392 [==============================] - 0s 238us/step - loss: 0.4631 - acc: 0.7959
Epoch 6/100
392/392 [==============================] - 0s 233us/step - loss: 0.4534 - acc: 0.7857
Epoch 7/100
392/392 [==============================] - 0s 258us/step - loss: 0.4578 - acc: 0.7857
Epoch 8/100
392/392 [==============================] - 0s 241us/step - loss: 0.4470 - acc: 0.7908
Epoch 9/100
392/392 [==============================] - 0s 257us/step - loss: 0.4344 - acc: 0.7908
Epoch 10/100
392/392 [==============================] - 0s 246us/step - loss: 0.4242 - acc: 0.8087
Epoch 11/100
392/392

In [ ]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.748, total=  13.4s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.4s remaining:    0.0s


[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.779, total=  13.0s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   26.3s remaining:    0.0s


...

[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.831, total=  13.3s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.0min remaining:    0.0s


[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.725, total=  13.8s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.756, total=  14.2s
[CV] dropout_rate=0.1, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.1, learn_rate=0.001, score=0.838, total=  14.6s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.725, total=  14.2s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.779, total=  14.6s
[CV] dropout_rate=0.1, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.1, learn_rate=0.01, score=0.808, total=  14.7s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.1, learn_rate=0.1, score=0.725, total=  15.0s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  6.6min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.7933673562139881, using {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7933673562139881 (0.04439222867892353) with: {'dropout_rate': 0.0, 'learn_rate': 0.001}
0.7704081603763054 (0.046457982913339614) with: {'dropout_rate': 0.0, 'learn_rate': 0.01}
0.7474489800479948 (0.05877389965595183) with: {'dropout_rate': 0.0, 'learn_rate': 0.1}
0.7729591880829967 (0.04779843308896438) with: {'dropout_rate': 0.1, 'learn_rate': 0.001}
0.7704081649378854 (0.03415914920363419) with: {'dropout_rate': 0.1, 'learn_rate': 0.01}
0.7244897942457881 (0.013377135648407042) with: {'dropout_rate': 0.1, 'learn_rate': 0.1}
0.785714290123813 (0.044365702353837765) with: {'dropout_rate': 0.2, 'learn_rate': 0.001}
0.7908163269867703 (0.037010630646375026) with: {'dropout_rate': 0.2, 'learn_rate': 0.01}
0.7193877520609875 (0.0158546377144747) with: {'dropout_rate': 0.2, 'learn_rate': 0.1}


In [ ]:
# Do a grid search to optimize kernel initialization and activation functions
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=softmax, init=uniform ................................
[CV] .... activation=softmax, init=uniform, score=0.756, total=  17.8s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.8s remaining:    0.0s


[CV] .... activation=softmax, init=uniform, score=0.756, total=  18.1s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   35.8s remaining:    0.0s


...

[CV] ....... activation=softmax, init=zero, score=0.700, total=  18.5s
[CV] activation=relu, init=uniform ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.7min remaining:    0.0s


[CV] ....... activation=relu, init=uniform, score=0.733, total=  18.4s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.756, total=  18.6s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.823, total=  19.7s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.718, total=  18.7s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.771, total=  19.6s
[CV] activation=relu, init=normal ....................................
[CV] ........ activation=relu, init=normal, score=0.823, total=  19.1s
[CV] activation=relu, init=zero ......................................
[CV] .......... activation=relu, init=zero, score=0.611, total=  19.1s
[CV] activation=relu, init=zero ......................................
[CV] .

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 12.0min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.7933673531729348, using {'activation': 'tanh', 'init': 'normal'}
0.7755102136609505 (0.028087641954935023) with: {'activation': 'softmax', 'init': 'uniform'}
0.7295918416003792 (0.08860773018104312) with: {'activation': 'softmax', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'softmax', 'init': 'zero'}
0.7704081655460961 (0.038262796592707045) with: {'activation': 'relu', 'init': 'uniform'}
0.7704081637214641 (0.04305242085893215) with: {'activation': 'relu', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'relu', 'init': 'zero'}
0.7831632721484924 (0.02879958859349298) with: {'activation': 'tanh', 'init': 'uniform'}
0.7933673531729348 (0.032371719468446684) with: {'activation': 'tanh', 'init': 'normal'}
0.6683673458744068 (0.040922317011154695) with: {'activation': 'tanh', 'init': 'zero'}
0.7933673531729348 (0.037313655933021314) with: {'activation': 'linear', 'init': 'uniform'}
0.7908163291155076 (0.033706165

In [ ]:
# Do a grid search to find the optimal number of neurons in each hidden layer
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(neuron1, neuron2):
    # create model
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), refit = True, verbose = 10)
grid_results = grid.fit(X_standardized, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=4, neuron2=2 ............................................
[CV] ................ neuron1=4, neuron2=2, score=0.771, total=  23.1s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.1s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=2, score=0.763, total=  23.7s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   46.8s remaining:    0.0s


...

[CV] ................ neuron1=4, neuron2=8, score=0.831, total=  24.8s
[CV] neuron1=8, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  3.6min remaining:    0.0s


[CV] ................ neuron1=8, neuron2=2, score=0.763, total=  25.3s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.763, total=  25.5s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.838, total=  25.0s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.763, total=  27.1s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.763, total=  25.7s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ................ neuron1=8, neuron2=4, score=0.838, total=  26.6s
[CV] neuron1=8, neuron2=8 ............................................
[CV] ................ neuron1=8, neuron2=8, score=0.763, total=  26.1s
[CV] neuron1=8, neuron2=8 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 11.5min finished


Best: 0.7908163351976142, using {'neuron1': 16, 'neuron2': 2}
0.785714290123813 (0.02650267677503863) with: {'neuron1': 4, 'neuron2': 2}
0.7882653126607135 (0.019356087898682556) with: {'neuron1': 4, 'neuron2': 4}
0.785714290123813 (0.03173682706863349) with: {'neuron1': 4, 'neuron2': 8}
0.7882653141812402 (0.03535836258888925) with: {'neuron1': 8, 'neuron2': 2}
0.7882653111401869 (0.03535836473101593) with: {'neuron1': 8, 'neuron2': 4}
0.7857142931648663 (0.031736824926506764) with: {'neuron1': 8, 'neuron2': 8}
0.7908163351976142 (0.03897990025127174) with: {'neuron1': 16, 'neuron2': 2}
0.7908163306360342 (0.03370616199600475) with: {'neuron1': 16, 'neuron2': 4}
0.7857142931648663 (0.031736824926506764) with: {'neuron1': 16, 'neuron2': 8}


In [ ]:
# generate predictions with optimal hyperparameters
y_pred = grid.predict(X_standardized)

In [ ]:
print(y_pred.shape)

(392, 1)


In [ ]:
print(y_pred[:5])

[[0]
 [1]
 [0]
 [1]
 [1]]


In [ ]:
# Generate a classification report
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.7806122448979592
              precision    recall  f1-score   support

           0       0.81      0.89      0.84       262
           1       0.71      0.57      0.63       130

    accuracy                           0.78       392
   macro avg       0.76      0.73      0.74       392
weighted avg       0.77      0.78      0.77       392

